# 126 Project Traffic Simulator

Let the road be length $n$. Cars start at position 0 and reach their destination at position $n-1$. Each car has an exponential "clock" with parameter $\gamma$, and every time their clock goes off they move forward with probability $p$. If a car is directly in front of them, they do not move regardless. A car enters the road at position 0 at a rate of $Exp(\alpha)$ and exit the road at position $n-1$ with a rate of $Exp(\beta)$.

### Choose global variables:

In [1]:
from ipywidgets import interactive
from IPython.display import display
from IPython.display import clear_output

def set_params(n, alpha, beta, theta, gamma, p, seed):
    return [n, alpha, beta, theta, gamma, p, seed]

params = interactive(set_params, n=(1, 20), alpha=(0.0, 2.0), beta=(0.0, 2.0), theta=(0.0, 0.5), gamma=(0.0, 2.0), p=(0.0, 1.0), seed=(1, 999))
display(params)

interactive(children=(IntSlider(value=10, description='n', max=20, min=1), FloatSlider(value=1.0, description=…

### Set global variables:
(Note: will break if you set alpha = 0 due to 0 division error)
Remember to rerun this cell every time a slider is updated

In [2]:
n, alpha, beta, theta, gamma, p, seed = params.result

### Define particle and environment classes:

In [7]:
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output


class Particle:
    
    count = 0
    
    def __init__(self, time, gamma=gamma, p=p):
        Particle.count += 1
        self.id = Particle.count - 1
        self.gamma = gamma
        self.p = p
        self._position = 0
        
        self._time = time # time at which the particle went to this position
        
    def update_position(self, time, value=1):
        self._position += value
        
        self._time = time # update time
        
    def get_position(self):
        return self._position
        
class Environment:
    def __init__(self, n=n, alpha=alpha, beta=beta):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.particles = []
        self.occupied_positions = [0 for _ in range(n)]
        
    def run_simulation(self, time_len=60):
        Particle.count = 0 # slight issue where running repeated simulations wouldn't cause particle counts to reset
        
        sys.stdout.write("\r{}".format(self.occupied_positions))
        sys.stdout.flush()
        
        curr_time = 0
        time_lapse = np.random.exponential(scale=1/self.alpha)
        time.sleep(time_lapse)
        curr_time += time_lapse
        particle = Particle(curr_time, gamma=gamma, p=p)
        self.particles.append(particle)
        self.occupied_positions[0] = 1
        
        actions = [(curr_time, particle.id, particle.get_position())]
        entrances = 1
        exits = 0
        
        travel_times = [[0, None]]
        cell_speeds = []
        
        while curr_time < time_len:
            sys.stdout.write("\r{}".format(self.occupied_positions))
            sys.stdout.flush()
            
            # should we include alpha/beta in the merged_param if the first spot is occupied/the last spot is empty?
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + self.alpha + self.beta
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha, self.beta]) / merged_param
            particle = np.random.choice(self.particles + ['entrance', 'exit'], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                if self.occupied_positions[0]:
                    continue
                else:
                    new_particle = Particle(curr_time, gamma=gamma, p=p)
                    self.particles.append(new_particle)
                    self.occupied_positions[0] = 1
                    
                    actions += [(curr_time, new_particle.id, new_particle.get_position())]
                    entrances += 1
                    
                    travel_times.append([curr_time, None])
                    
            elif particle == 'exit':
                if self.occupied_positions[self.n-1]: # changed to refer to self.n instead of n
                    end_particle = self.particles.pop(0)
                    self.occupied_positions[self.n-1] = 0 # changed to refer to self.n instead of n
                    
                    actions += [(curr_time, end_particle.id, 'exit')]
                    exits += 1
                    
                    travel_times[end_particle.id][1] = curr_time
                    
                else:
                    continue
            else:
                particle_pos = particle.get_position()
                if particle_pos == self.n - 1 or self.occupied_positions[particle_pos + 1] == 1: # changed to refer to self.n instead of n
                    continue
                else:
                    self.occupied_positions[particle_pos] = 0
                    self.occupied_positions[particle_pos + 1] = 1
                    
                    cell_speeds.append(1 / (curr_time - particle._time))
                    particle.update_position(curr_time)
                    
                    actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        travel_times = [end - begin for begin, end in travel_times if end]
        if len(travel_times) > 0:
            print("Travel times: {}".format([round(t, 2) for t in travel_times]))
            print('Average travel time: {}'.format(sum(travel_times) / len(travel_times)))
        else:
            print('No cars reached the end')
        if len(cell_speeds) > 0:
            print('Average cell speed: {}'.format(sum(cell_speeds) / len(cell_speeds)))
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Add 2D Environment:

In [9]:
class Particle2D(Particle):
    def __init__(self, gamma=gamma, p=p, direction=0): # 0 is horizontal, 1 is vertical
        Particle.__init__(self, gamma, p)
        self._position = [-1, -1]
        self._direction = direction
        
    def update_position(self, value=1):
        self._position[self._direction] += value
        
    def set_position(self, value=[-1, -1]):
        self._position = value
        
    def get_direction(self):
        return self._direction
        
# FIRST INDEX COLUMN, SECOND INDEX ROAD
class Env2D(Environment):
    def __init__(self, num_roads=(1,1), n=n, alpha=alpha, beta=beta, theta=theta):
        Environment.__init__(self, n, alpha, beta)
        self.theta = theta
        self.occupied_positions = np.array([[0 for _ in range(n)] for _ in range(n)])
        self.horizontal_roads = np.random.choice(range(n), num_roads[0], replace=False)
        self.vertical_roads = np.random.choice(range(n), num_roads[1], replace=False)
        self.num_roads = num_roads
        self.intersections = []
        [[self.intersections.append((x, y)) for x in self.vertical_roads] for y in self.horizontal_roads]
        self.inter_directions = {}
        for i in self.intersections:
            self.inter_directions[i] = 0
        
    def run_simulation(self, time_len=60):
        Particle.count = 0 # slight issue where running repeated simulations wouldn't cause particle counts to reset
        
        print(self.occupied_positions)
        
        curr_time = 0
        time_lapse = np.random.exponential(scale=1/(self.alpha * sum(self.num_roads)))
        time.sleep(time_lapse)
        curr_time += time_lapse
        particle = Particle2D(gamma=gamma, p=p, direction=np.random.choice([0, 1], p=[self.num_roads[0]/sum(self.num_roads), self.num_roads[1]/sum(self.num_roads)]))
        self.particles.append(particle)
        if not particle.get_direction():
            road = np.random.choice(self.horizontal_roads)
            particle.set_position([0, road])
            self.occupied_positions[0, road] = 1
        else:
            road = np.random.choice(self.vertical_roads)
            particle.set_position([road, 0])
            self.occupied_positions[road, 0] = 1
        
        actions = [(curr_time, particle.id, particle.get_position())]
        entrances = 1
        exits = 0
        
        while curr_time < time_len:
            clear_output(wait=True)
            print(self.occupied_positions)
            
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + (self.alpha + self.beta) * sum(self.num_roads) + self.theta * (sum(self.num_roads) ** 2)  
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha for _ in range(sum(self.num_roads))] + [self.beta for _ in range(sum(self.num_roads))] + [self.theta for _ in range(sum(self.num_roads) ** 2)]) / merged_param
            particle = np.random.choice(self.particles + ['entrance' for _ in range(sum(self.num_roads))] + ['exit' for _ in range(sum(self.num_roads))] + ['intersection' for _ in range(sum(self.num_roads) ** 2)], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[0][road]:
                        continue
                    new_particle = Particle2D(gamma=gamma, p=p, direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[0][road] = 1
                    new_particle.set_position([0, road])
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][0]:
                        continue
                    new_particle = Particle2D(gamma=gamma, p=p, direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[road][0] = 1
                    new_particle.set_position([road, 0])
                
                    
                actions += [(curr_time, new_particle.id, new_particle.get_position())]
                entrances += 1
                    
            elif particle == 'exit':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[self.n-1][road]: # changed to refer to self.n instead of n
                        particle = [p for p in self.particles if p.get_position() == [self.n-1, road]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[self.n-1][road] = 0 # changed to refer to self.n instead of n
                        actions += [(curr_time, particle.id, [n, road])]
                        exits += 1
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][self.n-1]: # changed to refer to self.n instead of n
                        particle = [p for p in self.particles if p.get_position() == [road, self.n-1]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[road][self.n-1] = 0 # changed to refer to self.n instead of n
                        actions += [(curr_time, particle.id, [road, n])]
                        exits += 1
            elif particle == 'intersection':
                i = np.random.choice(range(len(self.intersections)))
                intersection = self.intersections[i]
                self.inter_directions[intersection] = 0 if self.inter_directions[intersection] else 1
                actions += [(curr_time, 'intersection' + str(intersection), self.inter_directions[intersection])] 
            else:
                particle_pos = particle.get_position()
                direction = particle.get_direction()
                if not direction:
                    pos = (particle_pos[0] + 1, particle_pos[1])
                    if pos in self.intersections and self.inter_directions[pos]:
                        continue
                    elif particle_pos[0] == self.n - 1 or self.occupied_positions[particle_pos[0] + 1, particle_pos[1]] == 1:
                        continue
                    else:
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0] + 1][particle_pos[1]] = 1
                        particle.update_position()
                        actions += [(curr_time, particle.id, particle.get_position())]
                else:
                    pos = (particle_pos[0], particle_pos[1] + 1)
                    if pos in self.intersections and not self.inter_directions[pos]:
                        continue
                    elif particle_pos[1] == self.n - 1 or self.occupied_positions[particle_pos[0], particle_pos[1] + 1] == 1:
                        continue
                    else:
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0]][particle_pos[1] + 1] = 1
                        particle.update_position()
                        actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Run simulator:

In [10]:
env = Env2D(num_roads=(3,3), n=n, alpha=alpha, beta=beta, theta=theta)
env.run_simulation()

[[1 0 0 0 0 0 0 1 1 0]
 [0 1 1 1 0 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [1 0 1 1 1 1 0 1 0 0]
 [1 1 1 1 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]

 done
Summary:
Total entrances: 28
Total exits: 2

 Log:
Time: 0.00042449186913516134
Particle: 0
New position: [1, 7]


Time: 0.09246667909384229
Particle: 1
New position: [0, 0]


Time: 0.10567576180769893
Particle: 2
New position: [6, 9]


Time: 0.13967186012356797
Particle: intersection(1, 0)
New position: 1


Time: 0.16741043160015048
Particle: intersection(7, 0)
New position: 1


Time: 0.4372943017048747
Particle: 3
New position: [7, 6]


Time: 0.4762105066856121
Particle: intersection(7, 0)
New position: 0


Time: 0.6145836469541406
Particle: 4
New position: [9, 7]


Time: 0.675832944000291
Particle: intersection(6, 7)
New position: 1


Time: 0.75955921414778
Particle: intersection(6, 0)
New position: 1


Time: 1.0752844264123826
Particle: intersectio

Time: 16.591343193753495
Particle: intersection(1, 8)
New position: 1


Time: 16.823059198443726
Particle: intersection(1, 7)
New position: 0


Time: 16.88873147815489
Particle: intersection(7, 0)
New position: 0


Time: 16.911396380372825
Particle: intersection(7, 0)
New position: 1


Time: 16.924434809392626
Particle: intersection(1, 0)
New position: 0


Time: 16.973384613899892
Particle: intersection(7, 0)
New position: 0


Time: 16.98589991377036
Particle: intersection(1, 7)
New position: 1


Time: 17.01149222049274
Particle: 3
New position: [7, 6]


Time: 17.145815151730286
Particle: intersection(6, 8)
New position: 0


Time: 17.253752330521497
Particle: intersection(7, 7)
New position: 0


Time: 17.286347692442504
Particle: intersection(7, 0)
New position: 1


Time: 17.31810453807774
Particle: 2
New position: [6, 9]


Time: 17.398122814652464
Particle: intersection(1, 7)
New position: 0


Time: 17.501152440468033
Particle: intersection(1, 7)
New position: 1


Time: 17.65546379963


Time: 32.224249460574605
Particle: 5
New position: [5, 8]


Time: 32.24018889646709
Particle: 10
New position: [6, 5]


Time: 32.40740705048237
Particle: intersection(1, 0)
New position: 1


Time: 32.40797793591656
Particle: 7
New position: [6, 7]


Time: 32.51107171643587
Particle: intersection(1, 8)
New position: 1


Time: 32.6438790674834
Particle: intersection(1, 8)
New position: 0


Time: 32.67497159033409
Particle: 8
New position: [1, 6]


Time: 32.78300910755635
Particle: intersection(1, 0)
New position: 0


Time: 32.851205019303784
Particle: 9
New position: [7, 4]


Time: 32.93574618651827
Particle: intersection(7, 8)
New position: 1


Time: 32.94954444817397
Particle: intersection(6, 0)
New position: 1


Time: 33.03947511088799
Particle: intersection(1, 7)
New position: 1


Time: 33.100976140735966
Particle: 6
New position: [7, 5]


Time: 33.155409533170115
Particle: 14
New position: [1, 5]


Time: 33.16217595065434
Particle: 17
New position: [0, 7]


Time: 33.17388981868965


Particle: intersection(7, 8)
New position: 1


Time: 51.2836059887713
Particle: intersection(6, 0)
New position: 1


Time: 51.30105804411023
Particle: intersection(1, 8)
New position: 0


Time: 51.41581119221426
Particle: intersection(7, 7)
New position: 1


Time: 51.77838852971581
Particle: intersection(7, 8)
New position: 0


Time: 51.807262793187064
Particle: intersection(1, 8)
New position: 1


Time: 51.89487343478614
Particle: intersection(7, 0)
New position: 1


Time: 51.98969703809307
Particle: intersection(1, 0)
New position: 0


Time: 52.00088056462069
Particle: intersection(1, 0)
New position: 1


Time: 52.24745186278303
Particle: intersection(1, 8)
New position: 0


Time: 52.273596062981404
Particle: intersection(7, 7)
New position: 0


Time: 52.54030459592678
Particle: 2
New position: [6, 10]


Time: 52.675583708912704
Particle: intersection(7, 8)
New position: 1


Time: 52.739069221575605
Particle: 21
New position: [1, 2]


Time: 52.90617269645077
Particle: intersection(6,

In [8]:
env1 = Environment(n=n, alpha=alpha, beta=beta)
env1.run_simulation(time_len = 10)

[1, 1, 0, 0, 0, 1, 1, 0, 0, 0]
 done
Summary:
Total entrances: 4
Total exits: 0
No cars reached the end
Average cell speed: 1.9036929917227903

 Log:
Time: 0.03600505178913028
Particle: 0
New position: 0


Time: 1.8162725700593534
Particle: 0
New position: 1


Time: 2.7606848057662425
Particle: 0
New position: 2


Time: 3.3055848341582315
Particle: 1
New position: 0


Time: 3.6670717241845
Particle: 0
New position: 3


Time: 3.771381497356743
Particle: 0
New position: 4


Time: 5.097603719240408
Particle: 1
New position: 1


Time: 6.144456290454906
Particle: 1
New position: 2


Time: 6.364621690156886
Particle: 0
New position: 5


Time: 6.742228455284331
Particle: 1
New position: 3


Time: 6.896405869430949
Particle: 2
New position: 0


Time: 7.04385505286863
Particle: 0
New position: 6


Time: 7.142916223849873
Particle: 2
New position: 1


Time: 8.529220008934969
Particle: 3
New position: 0


Time: 8.688690313060528
Particle: 1
New position: 4


Time: 9.77655003617445
Particle: 1
New